<a href="https://colab.research.google.com/github/eftekhar-hossain/Disaster_IEEE-Access/blob/main/damage_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
pip install seaborn --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 1.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 883.9 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
pip install -U scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 1.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json
from sklearn.metrics import confusion_matrix,classification_report 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from keras.utils.vis_utils import plot_model
np.random.seed(42)
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import string, spacy,unicodedata, random
from bs4 import BeautifulSoup
class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings
# pretrained_path = "/content/drive/MyDrive/TaheriThesis/Dataset/PreTrained Wordembedding/"
# dataset_path = "/content/drive/MyDrive/Colab Notebooks/MSC Presentation Tasks/Disaster Response/Images/"
# folder_path = "/content/drive/MyDrive/Colab Notebooks/MSC Presentation Tasks/Disaster Response/"
models_path = "/Users/zaima/Desktop/Dataset/Multimodal Sentiment/"


folder_path = "/Users/zaima/Desktop/Dataset/"
# models_path = "/content/drive/MyDrive/TaheriThesis/Dataset/Multimodal Sentiment/Models/"
# results_path = "/content/drive/MyDrive/TaheriThesis/Dataset/Multimodal Sentiment/Results/"
dataset_path = "/Users/zaima/Desktop/Dataset/Images/"

[nltk_data] Downloading package punkt to /Users/zaima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CPU times: user 5.79 s, sys: 779 ms, total: 6.57 s
Wall time: 16.6 s


In [6]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, GlobalAveragePooling2D, Embedding, LSTM, multiply
from keras.models import Model
from keras import preprocessing, Input
from tensorflow import keras
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
# from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras.models import load_model
import itertools
from PIL import Image, ImageFile
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D,MaxPooling1D,Conv1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Add, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Nadam
keras.backend.clear_session()

#Fetching Dataset

In [8]:
# train_data = pd.read_excel(folder_path+'train.xlsx')
# test_data = pd.read_csv(folder_path+'test.xlsx')
# valid_data = pd.read_excel(main_path+'valid_new.xlsx')
train_data = pd.read_csv(folder_path+'train.csv')
valid_data = pd.read_csv(folder_path+'val.csv')
test_data = pd.read_csv(folder_path+'test.csv')

In [9]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label'], dtype='object')

In [10]:
train_data['Label'].value_counts()

Label
0    861
6    716
4    623
1    477
5    408
3    400
2    348
Name: count, dtype: int64

In [11]:
test_data['Label'].value_counts()

Label
0    93
6    79
4    68
5    46
3    44
1    44
2    40
Name: count, dtype: int64

In [12]:
# train_data['enc_label'] = train_data['label'].replace({'non_damage':0,'damaged_infrastructure':1,'damaged_nature':2,
#                                                        'fires':3,'flood':4,'human_damage':5 })
# test_data['enc_label'] = test_data['label'].replace({'non_damage':0,'damaged_infrastructure':1,'damaged_nature':2,
#                                                      'fires':3,'flood':4,'human_damage':5 })


train_data['enc_label'] = train_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})
valid_data['enc_label'] = valid_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})



In [13]:
test_data['enc_label'] = test_data['Label_Sentiment'].replace({'happy':0,'angry':1,'disgust':2,'fear':3,'sad':4,'surprise':5,'other':6})

In [14]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the training data ==> ", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (3833, 5)
Size of the training data ==>  (414, 5)
Size of the Test data ==> (414, 5)


## Image Fetching

In [15]:
## collect image names from the folders
def create_img_array(img_dirct):
    all_imgs = []
    for root, j, files in os.walk(img_dirct):
        for file in files:
            file = root + '' + file
            all_imgs.append(file)
    return all_imgs

def create_img_path(DF, Col_name, img_dir):
    img_path = [img_dir + '' + name for name in DF[Col_name]]
    return img_path

In [16]:
# Creating train, test and validation image path
train_img_path = create_img_path(train_data,'image_name', dataset_path)
valid_img_path = create_img_path(valid_data,'image_name', dataset_path)


In [17]:
test_img_path = create_img_path(test_data,'image_name', dataset_path)

In [18]:
# Function that returns image reading from the path
def get_input(path):
    # Loading image from given path
    # and resizing it to 150*150*3 format
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    img= tf.keras.utils.load_img(path, target_size=(150,150)) 
    # img.close()   
    return(img)

# Takes in image and preprocess it
def process_input(img):
    # Converting image to array    
    img_data =tf.keras.utils.img_to_array(img)
    # Adding one more dimension to array    
    img_data = np.expand_dims(img_data, axis=0)
    #     
    img_data = preprocess_input(img_data)
    #img_data = preprocess_input(img_data)
    return(img_data)


In [19]:
# Create an array of training images
train_images = []
for n,i in enumerate(train_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  train_images.append(process_img[0])

In [20]:
# Create an array of training images
valid_images = []
for n,i in enumerate(valid_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  valid_images.append(process_img[0])

In [21]:
# Create an array of training images
test_images = []
for n,i in enumerate(test_img_path):
  input_img = get_input(i)
  process_img = process_input(input_img)
  # print(n)
  test_images.append(process_img[0])


In [22]:
# convert into numpy array
train_image = np.array(train_images)
# convert into numpy array


In [23]:
valid_image = np.array(valid_images)

In [24]:
test_image = np.array(test_images)

In [25]:
import pickle as pkl
with open(dataset_path+'train.pkl','wb') as f:
    pkl.dump(train_image, f)

# with open(dataset_path+'test.pkl','wb') as f:
#     pkl.dump(test_image, f)    

In [26]:
import pickle as pkl
with open(dataset_path+'valid.pkl','wb') as f:
    pkl.dump(valid_image, f)

In [27]:
import pickle as pkl


with open(dataset_path+'test.pkl','wb') as f:
    pkl.dump(test_image, f)    

In [28]:
import pickle as pkl

with open(dataset_path+'train.pkl','rb') as f:
  train_image = pkl.load(f)
  print("Training Images:-- ",train_image.shape)

    

Training Images:--  (3833, 150, 150, 3)


In [29]:

with open(dataset_path+'valid.pkl','rb') as f:
  valid_image = pkl.load(f)
  print("Validation Images:-- ",valid_image.shape)

Validation Images:--  (414, 150, 150, 3)


In [30]:
with open(dataset_path+'test.pkl','rb') as f:
  test_image = pkl.load(f)
  print("Test Images:-- ",test_image.shape)

Test Images:--  (414, 150, 150, 3)


## One Hot Encoding

In [32]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [33]:
# encoder=OneHotEncoder(sparse=False)

# encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['label']]))

# encoded_labels .columns = encoder.get_feature_names(['label'])

# train_data= pd.concat([train_data, encoded_labels ], axis=1)

import pandas as pd

# Define the list of categories for your Label_Sentiment variable
categories = ['happy', 'angry', 'disgust', 'fear', 'sad', 'surprise', 'other']

# Perform one-hot encoding using get_dummies() function
encoded_labels = pd.get_dummies(train_data['Label_Sentiment'], columns=categories, prefix='Label_Sentiment')

# Concatenate the encoded labels with your original train_data DataFrame
train_data = pd.concat([train_data, encoded_labels], axis=1)

In [34]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label', 'enc_label',
       'Label_Sentiment_angry', 'Label_Sentiment_disgust',
       'Label_Sentiment_fear', 'Label_Sentiment_happy',
       'Label_Sentiment_other', 'Label_Sentiment_sad',
       'Label_Sentiment_surprise'],
      dtype='object')

In [35]:
# train_data = train_data.rename(columns={'label_damaged_infrastructure':'damaged_infrastructure', 'label_damaged_nature':'damaged_nature', 
#                                         'label_fires':'fires','label_flood':'flood',
#                                         'label_human_damage':'human_damage', 'label_non_damage':'non_damage'})


train_data = train_data.rename(columns={'Label_Sentiment_happy':'happy', 
                                        'Label_Sentiment_angry':'angry',
                                        'Label_Sentiment_disgust':'disgust',
                                        'Label_Sentiment_fear':'fear',
                                        'Label_Sentiment_sad':'sad',
                                        'Label_Sentiment_surprise':'surprise',
                                        'Label_Sentiment_other':'other'})
     

train_data

,image_name,Captions,Label_Sentiment,Label,enc_label,angry,disgust,fear,happy,other,sad,surprise
0,Screenshot 2023-04-20 at 3.34.16 PM.png,আমি সম্পূর্ণ অবাক হয়েছিলাম যখন আমি দেখি এটি ফ...,surprise,5,5,False,False,False,False,False,False,True
1,996.jpg,এখানে ভালবাসা অনুভব করুন! ভালবাসা দিবস,happy,0,0,False,False,False,True,False,False,False
2,992.jpg,"সত্যি কথা বলতে, সবকিছু আমাকে চিৎকার করতে চায়।",angry,1,1,True,False,False,False,False,False,False
3,99123189_2622639137993646_7342319916193677312_...,আশ্চর্যজনকভাবে ছবিটার সাথে বর্তমান বিশ্ব পরিস্...,surprise,5,5,False,False,False,False,False,False,True
4,990.jpg,খুব যত্নশীল হট হট হট হট হট হট হট হট - আমরা আরও...,angry,1,1,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4138,10015063_288553201304626_62844069571247761_o.jpg,এখন আমার মনে হচ্ছে আমরা দূরে সরে যাচ্ছি...... ...,sad,4,4,False,False,False,False,False,True,False
4139,1001.jpg,জো এর প্রথম প্রেম,happy,0,0,False,False,False,True,False,False,False
4140,1000.jpg,চিত্রগ্রহণের প্রথম দিন #পাওয়ারলেস ব্যাক ২০১১ ...,happy,0,0,False,False,False,True,False,False,False
4141,100.jpg,এটি আমার ভ্যালেন্টাইন এর ১ জন ভাগ্নের। আমি আনন...,happy,0,0,False,False,False,True,False,False,False


In [36]:
train_data.columns

Index(['image_name', 'Captions', 'Label_Sentiment', 'Label', 'enc_label',
       'angry', 'disgust', 'fear', 'happy', 'other', 'sad', 'surprise'],
      dtype='object')

In [37]:
train_data.Captions = train_data.Captions.astype(str)

##Evaluation Metrics

In [38]:
'''Evaluation Parameters'''

def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

## Confusion Matrix

In [39]:
## Confusion matrix function
def con_mat(true,pred,class_names,model_name):
  cm = confusion_matrix(true,pred)
  #sns.set()
  plt.figure(figsize =(4, 3))
  ax = plt.subplot()
 # 'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            #'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
  sns.heatmap(cm, annot=True,fmt="d",cmap='YlOrRd', ax = ax,annot_kws={"size": 10},) #annot=True to annotate cells
  # labels, title and ticks
  ax.set_xlabel('Predicted labels',fontsize=10)
  ax.set_ylabel('True labels',fontsize=10) 
  #ax.set_title(f'Confusion Matrix of {model_name}',fontsize=10) 
  ax.xaxis.set_ticklabels(class_names, rotation=45); ax.yaxis.set_ticklabels(class_names, rotation=45);
  ax.xaxis.tick_top()
 # plt.savefig(folder_path + f"{model_name}.png",bbox_inches='tight',dpi =500)

In [40]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_data['enc_label']),
                                        y = train_data['enc_label']                                                   
                                    )
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))
class_weights

{0: 0.6377770935960592,
 1: 1.1360021935837674,
 2: 1.5867483722711604,
 3: 1.3796203796203796,
 4: 0.8900107411385607,
 5: 1.3270339525944908,
 6: 0.7578196451435888}

## Callbacks

In [41]:
''' Callbacks'''
keras.backend.clear_session()
def callbacks_check(model_name):
  num_classes = 7
  accuracy_threshold = 0.99

  class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>accuracy_threshold):
          print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
          self.model.stop_training = True

  acc_callback = myCallback()
  # Saved the Best Model
  filepath = models_path+'Models_3/'+f"{model_name}.h5"
  checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
  # callback list
  callback_list = [acc_callback, checkpoint] 

  return callback_list

In [42]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(train_data['enc_label'])
y_valid=to_categorical(valid_data['enc_label'])

In [43]:
from tensorflow.keras.utils import to_categorical
y_test=to_categorical(test_data['enc_label'])

In [44]:
y_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

#Visual Models

In [45]:
from keras.applications.densenet import DenseNet121

In [53]:
keras.backend.clear_session()

def visual_models():

  # create the base pre-trained model
  vgg = VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  for layer in vgg.layers:
      layer.trainable = False
  # add a global spatial average pooling layer
  x = vgg.output
  pool = GlobalAveragePooling2D()(x)
  
  # image_flat = Flatten()(x)
  # image_dense = Dense(128, activation='relu')(image_flat)
  # image_drop = Dropout(0.2)(x)
  # let's add a fully-connected layer
  #flat = Flatten()(pool)
  # and a logistic layer -- let's say we have 200 classes
  #hidden1 = Dense(512, activation='relu')(flat)
  #dropout = Dropout(0.1)(hidden1)
  output = Dense(7, activation='softmax')(pool)
  # this is the model we will train
  vgg_img_model = Model(inputs=vgg.input, outputs=output)

 


  ##inception
  inception = keras.applications.InceptionV3(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  for layer in inception.layers:
      layer.trainable = False
  # add a global spatial average pooling layer
  z = inception.output
  pool2 = GlobalAveragePooling2D()(z)
  output2 = Dense(7, activation='softmax')(pool2)
  # this is the model we will train
  inception_img_model = Model(inputs=inception.input, outputs=output2)



 
 

  models = [vgg_img_model,inception_img_model]

  return models


In [54]:
model_name = ['vgg16','inception']
list_model = visual_models()

for mn,model in enumerate(list_model):
  if model == 'vgg16':
      opt = "RMSprop"
  else:
      opt = "adam" 
  model.compile(loss='categorical_crossentropy', 
                      optimizer=opt
, 
                      metrics = ["accuracy"])
  print(f"Model Name: {model_name[mn]}\n")
  # Training model
  model.fit(x=train_image, 
              y=y_train,
              epochs=51, 
              batch_size =32,
              validation_data=(valid_image,y_valid),
              verbose = 1,
              class_weight = class_weights,
              callbacks = callbacks_check(model_name[mn])
              )

Model Name: vgg16

Epoch 1/51
130/130 [==============================] - ETA: 0s - loss: 5.8832 - accuracy: 0.1661
Epoch 1: val_accuracy improved from -inf to 0.20482, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/vgg16.h5
130/130 [==============================] - 72s 526ms/step - loss: 5.8832 - accuracy: 0.1661 - val_loss: 4.5847 - val_accuracy: 0.2048
Epoch 2/51
130/130 [==============================] - ETA: 0s - loss: 3.9774 - accuracy: 0.2155
Epoch 2: val_accuracy improved from 0.20482 to 0.20964, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/vgg16.h5
130/130 [==============================] - 66s 506ms/step - loss: 3.9774 - accuracy: 0.2155 - val_loss: 3.7069 - val_accuracy: 0.2096
Epoch 3/51
130/130 [==============================] - ETA: 0s - loss: 3.1507 - accuracy: 0.2534
Epoch 3: val_accuracy improved from 0.20964 to 0.26265, saving model to /Users/zaima/Desktop/Dataset/Multimodal Sentiment/Models_3/vgg16.h5
130/130 

## Visual Models Performance

In [18]:
from sklearn.metrics import matthews_corrcoef
visual_models = ['vgg16','inception']
visual_model_names = ['VGG16','Inception']

def visual_models_accuracy(saved_model): 
  my_dict = {}
  # Prediction 
  model = load_model(models_path+'Models_3/'+f"{saved_model}.h5")
  y_pred = np.argmax(model.predict(test_image), axis=-1)

  y_true = test_data['enc_label']

  my_dict['Accuracy'] = accuracy_score(y_true, y_pred)*100
  my_dict['Precision'] = precision_score(y_true, y_pred,average = 'weighted')*100
  my_dict['Recall'] = recall_score(y_true, y_pred,average = 'weighted')*100 
  my_dict['F1 Score'] = f1_score(y_true, y_pred,average = 'weighted')*100 
  my_dict['MCC'] = matthews_corrcoef(y_true, y_pred)*100
  
  return my_dict

In [19]:
accuracy = {f'{visual_model_names[i]}':visual_models_accuracy(model) for i,model in enumerate(visual_models)}
# Save the performance parameter into json file
with open(models_path+'Results_3/'+'visual_models_performance.json', 'w') as f:
    json.dump(accuracy, f)


2023-06-11 20:01:22.599817: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


13/13 [==============================] - 3s 173ms/step


In [4]:
# Load the json file
metrics = json.load(open(models_path+'Results_3/'+'visual_models_performance.json'))
acc_list = []
pr_list = []
re_list = []
f1_list = []
mcc_list = []
for i in metrics.keys():
  acc_list.append(round(metrics[i]['Accuracy'],2))
  pr_list.append(round(metrics[i]['Precision'],2))
  re_list.append(round(metrics[i]['Recall'],2))
  f1_list.append(round(metrics[i]['F1 Score'],2))
  mcc_list.append(round(metrics[i]['MCC'],2))

print (color.BOLD+f"=======  Visual Models Performance on Test Data  =============\n"+color.END)
# Create a dataframe
performance_matrix = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list,'MCC':mcc_list},
                                  index =['VGG16','Inception'])
performance_matrix

=======  Visual Models Performance on Test Data  =============



,Accuracy,Precision,Recall,F1 Score,MCC
VGG16,70.44,71.75,68.44,70.06,63.15
Inception,71.62,72.09,68.62,70.31,64.23
